# Test CSW QUERY

the following will run a query by posting an  XML file via CURL 


At the moment the following Query are supported:
* Group A
    - `anytext`
    - `Time`
    - `BBOX`
    - `anytext` *on field*

* Group B
    - `anytext` **AND** `Time`
    - `anytext` **AND** `BBOX`
    - `anytext` **AND** `Time` **AND** `BBOX`
  
* Group C
    - `anytext` *on field*  **AND** `Time`
    - `anytext` *on field*  **AND** `BBOX`
    - `anytext` *on field* **AND** `Time` **AND** `BBOX`

* Group D
    - `Time` **AND** `BBOX`
    
* Group E
    - `anytext` *on field*  **AND** `BBOX` OR `anytext` *on field*  **AND** `BBOX`

In [ ]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [ ]:
csw_test_endpoint = "https://test.csw.met.no"
ElementSetName = "brief"

field_to_query = "title"
text_to_quey = "ice"

start_time = "2018-09-18 09:00"
end_time = "2020-10-18 09:00"

lowerCorner = "60 0"
upperCorner = "90 180"

## Group A

### `anytext`

* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

### `Time`


* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_time.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
     <ogc:Filter>
       <ogc:And>
         <ogc:PropertyIsLessThanOrEqualTo>
           <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
           <ogc:Literal>{end_time}</ogc:Literal>
         </ogc:PropertyIsLessThanOrEqualTo>
        <ogc:PropertyIsGreaterThanOrEqualTo>
           <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
           <ogc:Literal>{start_time}</ogc:Literal>
         </ogc:PropertyIsGreaterThanOrEqualTo>
       </ogc:And>
     </ogc:Filter>
   </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_time.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

### `BBOX`



* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_bbox.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:BBOX>
                    <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                    <gml:Envelope>
                        <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                        <gml:upperCorner>{upperCorner}</gml:upperCorner>
                    </gml:Envelope>
                </ogc:BBOX>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_bbox.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

### `anytext` *on field*



* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_on_field.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

## Group B

### `anytext` **AND** `Time`

* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_AND_time.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_AND_time.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

### `anytext` **AND** `BBOX`


* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

### `anytext` **AND** `Time` **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_AND_time_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                        <gml:Envelope>
                            <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                            <gml:upperCorner>{upperCorner}</gml:upperCorner>
                        </gml:Envelope>
                    </ogc:BBOX>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_AND_time_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

## Group C

### `anytext` *on field*  **AND** `Time`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_on_field_AND_time.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_time.xml {csw_test_endpoint}  | grep -oP 'numberOfRecordsMatched="\K[^"]+'

### `anytext` *on field*  **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_on_field_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>full</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

### `anytext` *on field* **AND** `Time` **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_on_field_AND_Time_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_quey}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_Time_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

## Group D

### `Time` **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_Time_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_Time_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

# Note



## PropertyIsEqualTo vs PropertyIsEqualTo

* Anytext can be used with `PropertyIsLike` or `PropertyIsEqualTo`
  when using `PropertyIsEqualTo` it requires `PropertyName` to be set:

  ```xml
              <ogc:PropertyIsEqualTo>
               <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
               <ogc:Literal>{text_to_quey}</ogc:Literal>
            </ogc:PropertyIsEqualTo>
  ```

In [ ]:
%%writetemplate test_xml_query_anytext_ofield_equal_to.xml
<?xml version="1.0" encoding="UTF-8"?>
<csw:GetRecords xmlns:csw="http://www.opengis.net/cat/csw/2.0.2"
               xmlns:ogc="http://www.opengis.net/ogc"
               xmlns:gml="http://www.opengis.net/gml"
               xmlns:ogc_filter="urn:x-ogc:def:filter:OGC:1.0:equals"
               service="CSW" version="2.0.2"
               resultType="results" startPosition="1" maxRecords="10">
   <csw:Query typeNames="csw:Record">
      <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
      <csw:Constraint version="1.1.0">
         <ogc:Filter>
            <ogc:PropertyIsEqualTo>
               <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
               <ogc:Literal>{text_to_quey}</ogc:Literal>
            </ogc:PropertyIsEqualTo>
         </ogc:Filter>
      </csw:Constraint>
   </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_ofield_equal_to.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

## OR Query 
OR query inside AND are not 'yet' supported, however, the case is identified in the code
I added few debug statments to detect such conditions
It could be useful to identify multiple TIME and BBOX query
At the moment if a multiple BBOX/TIME is provided the code will silently IGNORE such constraint

One more query from Lassi:

If I send a query with this Filter, which selects records from certain area and containing text 'area' in title:




* Corresponding SOLR JSON Query:

```json
{
    "q": "(title:temperature && _query_:\"{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))\") OR (title:area && _query_:\"{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))\")",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "collection:(ADC)"
    ]
}
```

In [ ]:
%%writetemplate test_xml_query_anytext_on_field_AND_BBOX_OR_testing_1.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>full</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>area</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_BBOX_OR_testing_1.xml {csw_test_endpoint}  | grep -oP 'numberOfRecordsMatched="\K[^"]+'


... I get back 1 record.

If I do the same with text 'temperature':

In [ ]:
%%writetemplate test_xml_query_anytext_on_field_AND_BBOX_OR_testing_2.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>brief</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>temperature</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_BBOX_OR_testing_2.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'


... I get back 3 records.

However, when I combine these two with 'Or':



In [ ]:
%%writetemplate test_xml_query_anytext_on_field_AND_BBOX_OR_testing_3.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>brief</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:Or>
                    <ogc:And>
                        <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                            <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                            <ogc:Literal>temperature</ogc:Literal>
                        </ogc:PropertyIsLike>
                        <ogc:BBOX>
                            <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                                <gml:Envelope>
                                    <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                    <gml:upperCorner>{upperCorner}</gml:upperCorner>
                                </gml:Envelope>
                        </ogc:BBOX>
                    </ogc:And>
                    <ogc:And>
                        <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                            <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                            <ogc:Literal>area</ogc:Literal>
                        </ogc:PropertyIsLike>
                        <ogc:BBOX>
                            <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                                <gml:Envelope>
                                    <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                    <gml:upperCorner>{upperCorner}</gml:upperCorner>
                                </gml:Envelope>
                        </ogc:BBOX>
                    </ogc:And>
                </ogc:Or>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [ ]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_BBOX_OR_testing_3.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

I get 4 records (as expected it could have been 3 or 4)